In [2]:
import requests
from bs4 import BeautifulSoup
import json
from time import time

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed

/Users/Welcome/Library/Python/3.9/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [3]:
#utils.py part
TYPE_CONFIG = {
    "사회": "society",
    "정치": "politics",
    "경제": "economic",
    "국제": "foreign",
    "문화": "culture",
    "연예": "entertain",
    "스포츠": "sports",
    "IT": "digital",
    "칼럼": "editorial",
    "보도자료": "press",
}

In [42]:
#**kwars 이거써서 코드 깔끔하게
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

def has_next_btn(html) -> bool:
    next_btn = html.select("div.box_etc > div.paging_news > span.inner_paging > a.btn_page.btn_next")
    if len(next_btn) == 0:
        return False
    else:
        return True
    
def makeURL_naver(search = ""):
    print("makeURL_naver")
    
def set_params() -> dict:
    print({"params":"params"})

def makeURL_daum(search = "", period="", start_date="", end_date="") -> str:
    url = "https://search.daum.net/search?w=news&DA=STC&enc=utf8"
    period_url = ""
    search_url = ""
    
    if period != "": #[TODO] 올바른 period가 안들어왔을때 에러 핸들링
        period_url = "&period=%s" % (period)
        
        if period == "u": #기간 조정
            period_url = period_url + "&sd=%s&ed=%s" % (start_date, end_date)
    
    if search !="":
        search_url = "&q=%s" % (search)
    return url + period_url + search_url

def return_select_func(html, request_type) -> list:
    if request_type == "search":
        return html.select("div.cont_divider > ul.list_news > li div.wrap_cont > a")
    elif request_type == "normal":
        return html.select("div.box_etc > ul.list_news2.list_allnews > li div.cont_thumb > strong.tit_thumb > a")

def extract_title(url, request_type) -> bool:
    try:
        original_html = requests.get(url)
        html = BeautifulSoup(original_html.text, "html.parser")
        print(html)    
        title_list = return_select_func(html, request_type)
        
        print([title.get_text() for title in title_list])
        if len(title_list) == 0 or not has_next_btn(html):
            return False
    except:
        return False

    return True

def newsScrape(search):
    URL = makeURL_daum(search)
    THREAD_COUNT = 10
    page_num = 1
    
    with ThreadPoolExecutor(THREAD_COUNT) as executor:
        while(True):
            url = f"{URL}&page={str(page_num)}"
            future = executor.submit(extract_title, url, "search")
            page_num = page_num + 1
            break
            if not future.result():
                break
        
def newsType(newsType="", date=""):
    THREAD_COUNT = 10
    page_num = 1
    
    BASE_URL = "https://news.daum.net/breakingnews/"
    if newsType != "":
        URL = f"{BASE_URL}{TYPE_CONFIG[newsType]}"
    
    if date != "": 
        URL = f"{BASE_URL}?regDate={date}" 
        
    
    with ThreadPoolExecutor(THREAD_COUNT) as executor:
        while(True):
            url = f"{URL}&page={str(page_num)}"
            future = executor.submit(extract_title, url, "normal")
            page_num = page_num + 1
            break
            if not future.result():
                break


if __name__ == '__main__':        
    start = time()
    #newsType("경제", "20220720")
    newsScrape("우영우")
    print("time : ", time() - start)
    
    
#[TODO] TYPE_CONFIG에 들어있지 않는 타입 들어왔을때 에러 핸들링
#[TODO] 올바른 date가 아닐때 에러 핸들링

<script type="text/javascript">
location.replace( "https://ncaptcha.search.daum.net/captcha/show?redirect=http%3A%2F%2Fsearch.daum.net%2Fsearch%3Fw%3Dnews%26DA%3DSTC%26enc%3Dutf8%26q%3D%25EC%259A%25B0%25EC%2598%2581%25EC%259A%25B0%26page%3D1" );
</script>

[]
time :  0.06569194793701172
